In [ ]:
%pip install openai
%pip install langchain
%pip install chromadb
%pip install pypdf

In [28]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
OPENAI_API_BASE = os.environ['OPENAI_API_BASE']

In [29]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('pdf_data/893.pdf')
pages = loader.load_and_split()

In [30]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# chunk_overlap=0 表示没有重叠的字符
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(pages)
print(f'{len(split_docs)} split_docs')

35 split_docs


In [31]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

persist_directory = "chroma_storage_893"

if os.path.exists(persist_directory) != True:
    # 把分块内容处理成 embeddings
    vectorstore = Chroma.from_documents(
        split_docs, embeddings, persist_directory=persist_directory)
    # 结果持久化
    vectorstore.persist()

In [32]:
query = '这篇论文和 Transformer 的相关性是什么？'
# 加载保存在本地的文档数据
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)
# 查询与提问最相关的2个文档
search_docs = vectordb.similarity_search(query, 2)
print(search_docs)

Using embedded DuckDB with persistence: data will be stored in: chroma_storage_893


[Document(page_content='第 28 卷 / 第 4 期 / 2023  年4 月刘禹良，李鸿亮，白翔，金连文  \n浅析 ChatGPT ： 历史沿革 、应用现状及前景展望\nformer 的双向深层预训练模型 ，其模型参数首次超\n过了 3亿规模 （BERT -Large 约有 3.4个参数 ） ；同年 ，\nOpenAI 提出了生成式预训练  Transformer 模型 ——\nGPT（generative pre -training ） （Radford 等，2018 ） ，大大\n地推动了自然语言处理领域的发展 。2018 年，人工\n智能系统 OpenAI Five （Berner 等，2019 ）战胜了世界\n顶级的 Dota 2人类队伍 ，人工智能在复杂任务领域\n树立了一个新的里程碑 ；此后 ，Google DeepMind 团\n队提出的 AlphaFold （Jumper 等，2021 ）以前所未有的\n准确度超越了人类研究者在蛋白质结构预测上的表\n现，突破了人们对人工智能在生物学领域的应用的\n想象 。2019 年，一种人工智能系统 AlphaStar （Viny ⁃\nals等，2019 ）战胜了世界顶级的 StarCraft II 人类选\n手，为人工智能在复杂任务领域的未来发展提供了\n更加强有力的支持 。2020 年，随着 OpenAI GPT -3模\n型（Brown 等，2020 ） （模型参数约 1 750亿）的问世 ，\n在 众 多 自 然 语 言 处 理 （natural language processing ，\nNLP）任务中 ，人工智能均表现出超过人类水平的能\n力。2021 年1月，Google Brain 提出了 Switch Trans ⁃\nformer 模型（Fedus 等，2021 ） ，以高达 1.6万亿的参数\n量成为史上首个万亿级语言模型 ；同年 12月，谷歌\n还提出了 1.2亿参数的通用稀疏模型 GLaM （Du等，\n2022 ） ，在多个小样本学习任务的性能超过 GPT-3。\n2022 年2月，人工智能生成内容 （artifical intelligence \ngenerated content ，AIGC ）技 术 被 《MIT Technology \nRev

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
# 构建llm 
# temperature=0 意味着需要最保守的回答 减小模型产生幻觉的几率
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY,
                 model_name='gpt-3.5-turbo', api_base=OPENAI_API_BASE)

chain = load_qa_chain(llm, chain_type='stuff')
results = chain.run(input_documents=search_docs, question=query)
print(f'Q: {query}')
print(f'A: {results}')

Q: 这篇论文和 Transformer 的相关性是什么？
A: 这篇论文中提到了 ChatGPT 是一种基于认知计算和人工智能的语言模型，它使用了 Transformer 架构和 GPT，即生成式预训练技术。GPT 训练的模型是一种应用于自然语言处理（NLP）的模型，它通过使用多层 Transformer 来预测下一个单词的概率分布，以生成自然语言文本。因此，这篇论文和 Transformer 有很大的相关性，因为它们都是用于自然语言处理的模型，并且使用了 Transformer 架构。


In [34]:
import xml.etree.ElementTree as ET

speak = ET.Element('speak')
speak.set('xmlns', 'http://www.w3.org/2001/10/synthesis')
speak.set('xmlns:mstts', 'http://www.w3.org/2001/mstts')
speak.set('xmlns:emo', 'http://www.w3.org/2009/10/emotionml')
speak.set('version', '1.0')
speak.set('xml:lang', 'en-US')

voice = ET.SubElement(speak, 'voice')
voice.set('name', 'zh-TW-YunJheNeural')

prosody = ET.SubElement(voice, 'prosody')
prosody.set('rate', '20%')
prosody.set('pitch', '20%')
prosody.text = query + results

ET.dump(speak)
tree = ET.ElementTree(speak)
tree.write(persist_directory + '/SSML.xml', encoding='utf-8')

<speak xmlns="http://www.w3.org/2001/10/synthesis" xmlns:mstts="http://www.w3.org/2001/mstts" xmlns:emo="http://www.w3.org/2009/10/emotionml" version="1.0" xml:lang="en-US"><voice name="zh-TW-YunJheNeural"><prosody rate="20%" pitch="20%">这篇论文和 Transformer 的相关性是什么？这篇论文中提到了 ChatGPT 是一种基于认知计算和人工智能的语言模型，它使用了 Transformer 架构和 GPT，即生成式预训练技术。GPT 训练的模型是一种应用于自然语言处理（NLP）的模型，它通过使用多层 Transformer 来预测下一个单词的概率分布，以生成自然语言文本。因此，这篇论文和 Transformer 有很大的相关性，因为它们都是用于自然语言处理的模型，并且使用了 Transformer 架构。</prosody></voice></speak>
